<a href="https://colab.research.google.com/github/arikmsn/RealvsAI/blob/FromCollab/RealVsAI_Collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q openai gspread google-auth-httplib2 google-auth-oauthlib pillow

import openai
import gspread
from google.colab import auth, userdata
from google.auth import default
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
import io
from PIL import Image
import base64
import time

# הגדרות
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')  # קריאה מאובטחת מ-Secrets
SHEET_ID = "12AqgbSH2LPcEtZE8BsfyWzNebRhLL_j6F6xhrQFd7sM"
WORKSHEET_NAME = "גיליון1"

# אימות
auth.authenticate_user()
creds, _ = default()

# חיבור ל-Sheets ו-Drive
gc = gspread.authorize(creds)
sheet = gc.open_by_key(SHEET_ID).worksheet(WORKSHEET_NAME)
drive_service = build('drive', 'v3', credentials=creds)

# הגדרת OpenAI
client = openai.OpenAI(api_key=OPENAI_API_KEY)

# קריאת השורות
rows = sheet.get_all_values()
headers = rows[0]
data_rows = rows[1:]

real_url_idx = headers.index('real_image_url')
prompt_idx = headers.index('Prompt')

print(f"🚀 מעבד {len(data_rows)} תמונות עם GPT-4o Vision...\n")

for i, row in enumerate(data_rows, start=2):
    real_url = row[real_url_idx] if real_url_idx < len(row) else ""

    if not real_url or 'drive.google.com' not in real_url:
        print(f"⚠️  שורה {i}: אין URL תקין, מדלג\n")
        continue

    try:
        # חילוץ File ID
        if '/d/' in real_url:
            file_id = real_url.split('/d/')[1].split('/')[0]
        elif 'id=' in real_url:
            file_id = real_url.split('id=')[1].split('&')[0]
        else:
            print(f"⚠️  שורה {i}: לא הצלחתי לחלץ file ID\n")
            continue

        # הורדה מ-Drive
        print(f"📥 שורה {i}: מוריד תמונה...")
        request = drive_service.files().get_media(fileId=file_id)
        file_bytes = io.BytesIO()
        downloader = MediaIoBaseDownload(file_bytes, request)

        done = False
        while done is False:
            status, done = downloader.next_chunk()

        file_bytes.seek(0)
        img = Image.open(file_bytes)

        # המרה ל-base64 ל-OpenAI
        buffered = io.BytesIO()
        img.save(buffered, format="PNG")
        img_base64 = base64.b64encode(buffered.getvalue()).decode()

        # שליחה ל-GPT-4o Vision
        print(f"🤖 שורה {i}: מנתח עם GPT-4o Vision...")

        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": """Analyze this image and create a SHORT, CONCISE prompt (max 100 words) for AI image generation tools like DALL-E or Midjourney.

Include ONLY:
- Main subject and composition
- Lighting and colors
- Mood and style
- Camera angle

Be specific but brief. No explanations or formatting."""
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/png;base64,{img_base64}"
                            }
                        }
                    ]
                }
            ],
            max_tokens=300
        )

        generated_prompt = response.choices[0].message.content.strip()

        # שמירה ל-Sheet
        print(f"💾 שורה {i}: שומר prompt...")
        sheet.update_cell(i, prompt_idx + 1, generated_prompt)

        print(f"✅ שורה {i}: הושלם!\n")
        print(f"📝 Prompt: {generated_prompt[:80]}...\n")

        time.sleep(2)  # למנוע rate limiting

    except Exception as e:
        print(f"❌ שורה {i}: שגיאה - {str(e)}\n")
        continue

print("\n🎉 סיום! כל ה-Prompts נשמרו ב-Sheet!")
print("💰 עלות משוערת: ~$0.03-0.06 (6 תמונות)")


🚀 מעבד 6 תמונות עם GPT-4o Vision...

📥 שורה 2: מוריד תמונה...
🤖 שורה 2: מנתח עם GPT-4o Vision...
💾 שורה 2: שומר prompt...
✅ שורה 2: הושלם!

📝 Prompt: Retro bus parked in a dimly lit garage, front view partially obscured by a concr...

📥 שורה 3: מוריד תמונה...
🤖 שורה 3: מנתח עם GPT-4o Vision...
💾 שורה 3: שומר prompt...
✅ שורה 3: הושלם!

📝 Prompt: Close-up of a red and white striped Christmas ornament hanging on a green pine t...

📥 שורה 4: מוריד תמונה...
🤖 שורה 4: מנתח עם GPT-4o Vision...
💾 שורה 4: שומר prompt...
✅ שורה 4: הושלם!

📝 Prompt: Close-up of a human eye, face partially visible, focus on blue iris. Soft, natur...

📥 שורה 5: מוריד תמונה...
🤖 שורה 5: מנתח עם GPT-4o Vision...
💾 שורה 5: שומר prompt...
✅ שורה 5: הושלם!

📝 Prompt: Giraffe in African savanna viewed from safari vehicle, overcast sky, muted earth...

📥 שורה 6: מוריד תמונה...
🤖 שורה 6: מנתח עם GPT-4o Vision...
💾 שורה 6: שומר prompt...
✅ שורה 6: הושלם!

📝 Prompt: Close-up portrait of a lion with intense gaze, centered co

In [6]:
!pip install -q openai gspread google-auth-httplib2 google-auth-oauthlib

import openai
import gspread
from google.colab import auth, userdata
from google.auth import default
import time

# Settings
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
SHEET_ID = "12AqgbSH2LPcEtZE8BsfyWzNebRhLL_j6F6xhrQFd7sM"
WORKSHEET_NAME = "גיליון1"

# Authentication
auth.authenticate_user()
creds, _ = default()

# Connect to Sheets
gc = gspread.authorize(creds)
sheet = gc.open_by_key(SHEET_ID).worksheet(WORKSHEET_NAME)

# Setup OpenAI
client = openai.OpenAI(api_key=OPENAI_API_KEY)

# Read rows
rows = sheet.get_all_values()
headers = rows[0]
data_rows = rows[1:]

prompt_idx = headers.index('Prompt')
ai_image_url_idx = headers.index('ai_image_url')

print(f"🎨 Generating 6 AI images with DALL-E 3...\n")

for i, row in enumerate(data_rows, start=2):
    prompt = row[prompt_idx] if prompt_idx < len(row) else ""

    if not prompt or len(prompt) < 10:
        print(f"⚠️  Row {i}: No prompt, skipping\n")
        continue

    try:
        print(f"🎨 Row {i}: Generating image...")
        print(f"📝 Prompt: {prompt[:70]}...\n")

        # Generate image with DALL-E 3
        response = client.images.generate(
            model="dall-e-3",
            prompt=prompt,
            size="1024x1024",
            quality="standard",
            n=1,
        )

        image_url = response.data[0].url

        # Update Sheet with URL
        print(f"💾 Row {i}: Saving URL to Sheet...")
        sheet.update_cell(i, ai_image_url_idx + 1, image_url)

        print(f"✅ Row {i}: Complete!\n")
        print(f"🔗 URL: {image_url}\n")

        time.sleep(2)

    except Exception as e:
        print(f"❌ Row {i}: Error - {str(e)}\n")
        continue

print("\n🎉 Done! All AI images generated and saved!")
print("💰 Estimated cost: ~$0.24 (6 images × $0.04)")


🎨 Generating 6 AI images with DALL-E 3...

🎨 Row 2: Generating image...
📝 Prompt: Retro bus parked in a dimly lit garage, front view partially obscured ...

💾 Row 2: Saving URL to Sheet...
✅ Row 2: Complete!

🔗 URL: https://oaidalleapiprodscus.blob.core.windows.net/private/org-cHKdOZCcVFWRvBe669zQOl9m/user-ZM2HtSLVxXSeVYdhfeHBHIXs/img-ZUlBPZWt49dI5iNgfCszug0F.png?st=2026-02-17T05%3A47%3A45Z&se=2026-02-17T07%3A47%3A45Z&sp=r&sv=2026-02-06&sr=b&rscd=inline&rsct=image/png&skoid=f1dafa11-a0c2-4092-91d4-10981fbda051&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2026-02-16T08%3A52%3A44Z&ske=2026-02-17T08%3A52%3A44Z&sks=b&skv=2026-02-06&sig=8EOyA/Z0p%2BJYo%2BKgKAvbEk7I7lRLGp4ofmOZdC/Lif0%3D

🎨 Row 3: Generating image...
📝 Prompt: Close-up of a red and white striped Christmas ornament hanging on a gr...

💾 Row 3: Saving URL to Sheet...
✅ Row 3: Complete!

🔗 URL: https://oaidalleapiprodscus.blob.core.windows.net/private/org-cHKdOZCcVFWRvBe669zQOl9m/user-ZM2HtSLVxXSeVYdhfeHBHIXs/img-VAUf33lHRtRb